In [1]:
from prody import *
from pylab import *

In [2]:
pdb, header = parsePDB('6acg', header=True)

@> PDB file is found in working directory (6acg.pdb).
@> 29715 atoms and 1 coordinate set(s) were parsed in 0.27s.


In [3]:
spike_contacts = pdb.select('chain A B C and within 4 of chain D')

In [4]:
ace2_contacts = pdb.select('chain D and within 4 of chain A B C')

In [5]:
# writePDB('spike_contacts.pdb', spike_contacts)
# writePDB('ace2_contacts.pdb', ace2_contacts)

In [6]:
spike = pdb.select('chain A B C')
ace2  = pdb.select('chain D')

In [7]:
interactions = findNeighbors(atoms=spike, radius=4, atoms2=ace2)

In [11]:
spike_atoms = []
ace2_atoms = []
links = []

In [12]:
def getAtomData(atom):
    data = {}

    data["chain"]   = atom.getData('chain')
    data["resname"] = atom.getData('resname')
    data["resnum"]  = int(atom.getData('resnum'))
    data["element"] = atom.getData('element')
    data["index"]   = atom.getIndex()

    return data

In [14]:
for pair in interactions:
    a, b, distance = pair

    spike_atom = getAtomData(a)    
    ace2_atom  = getAtomData(b)

    spike_atoms.append(spike_atom)
    ace2_atoms.append(ace2_atom)

    links.append({
        "source": spike_atom["index"],
        "target": ace2_atom["index"]
    })

In [19]:
spike_atoms_uniq = list({v['index']:v for v in spike_atoms}.values())
ace2_atoms_uniq  = list({v['index']:v for v in ace2_atoms}.values())

In [23]:
interaction_json = {
    "nodes": spike_atoms_uniq + ace2_atoms_uniq,
    "links": links
}

In [ ]:
with open('interactions.json', 'w') as file:
    json.dump(interaction_json, file